## Country Club queries, Part 2

Let's first prepare the database access, and list tables from the database:

In [2]:
from sqlalchemy import create_engine

# Create engine and open engine connection
# To connect to a local file, see: 
# https://docs.sqlalchemy.org/en/13/core/engines.html#sqlite
#
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')
con = engine.connect()

print(engine.table_names())

['Bookings', 'Facilities', 'Members']


To recall the table content. let's print out the first few entries of those tables:

In [30]:
rs = con.execute("SELECT * FROM Bookings LIMIT 5")
print(rs.keys())
rs.fetchall()

['bookid', 'facid', 'memid', 'starttime', 'slots']


[(0, 3, 1, '2012-07-03 11:00:00', 2),
 (1, 4, 1, '2012-07-03 08:00:00', 2),
 (2, 6, 0, '2012-07-03 18:00:00', 2),
 (3, 7, 1, '2012-07-03 19:00:00', 2),
 (4, 8, 1, '2012-07-03 10:00:00', 1)]

In [29]:
rs = con.execute("SELECT * FROM Facilities LIMIT 5")
print(rs.keys())
rs.fetchall()

['facid', 'name', 'membercost', 'guestcost', 'initialoutlay', 'monthlymaintenance']


[(0, 'Tennis Court 1', 5, 25, 10000, 200),
 (1, 'Tennis Court 2', 5, 25, 8000, 200),
 (2, 'Badminton Court', 0, 15.5, 4000, 50),
 (3, 'Table Tennis', 0, 5, 320, 10),
 (4, 'Massage Room 1', 9.9, 80, 4000, 3000)]

In [35]:
rs = con.execute("SELECT * FROM Members LIMIT 5")
print(rs.keys())
rs.fetchall()

['memid', 'surname', 'firstname', 'address', 'zipcode', 'telephone', 'recommendedby', 'joindate']


[(0, 'GUEST', 'GUEST', 'GUEST', 0, '(000) 000-0000', '', '2012-07-01 00:00:00'),
 (1, 'Smith', 'Darren', '8 Bloomsbury Close, Boston', 4321, '555-555-5555', '', '2012-07-02 12:02:05'),
 (2, 'Smith', 'Tracy', '8 Bloomsbury Close, New York', 4321, '555-555-5555', '', '2012-07-02 12:08:23'),
 (3, 'Rownam', 'Tim', '23 Highway Way, Boston', 23423, '(844) 693-0723', '', '2012-07-03 09:32:15'),
 (4, 'Joplette', 'Janice', '20 Crossing Road, New York', 234, '(833) 942-4710', '1', '2012-07-03 10:25:05')]

Cool. Now we're ready to answer the questions. 

Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [59]:
query="""with cte as (SELECT name,
CASE WHEN memid = 0 THEN slots * guestcost
ELSE slots * membercost END AS cost
FROM Bookings
INNER JOIN Facilities
USING ( facid ) )

SELECT name as facility_name, sum(cost) as revenue
FROM cte
GROUP BY name
HAVING(revenue < 1000)
ORDER BY revenue
"""
rs = con.execute(query)
print(rs.keys())
rs.fetchall()

['facility_name', 'revenue']


[('Table Tennis', 180), ('Snooker Table', 240), ('Pool Table', 270)]

Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [64]:
query="""SELECT m1.surname,m1.firstname, 
         m2.surname || " " || m2.firstname as recommended_by
FROM Members as m1
LEFT JOIN Members as m2
ON m1.recommendedby = m2.memid
WHERE m1.surname <> "GUEST"
ORDER BY m1.surname,m1.firstname
"""
rs = con.execute(query)
print(rs.keys())
rs.fetchall()

['surname', 'firstname', 'recommended_by']


[('Bader', 'Florence', 'Stibbons Ponder'),
 ('Baker', 'Anne', 'Stibbons Ponder'),
 ('Baker', 'Timothy', 'Farrell Jemima'),
 ('Boothe', 'Tim', 'Rownam Tim'),
 ('Butters', 'Gerald', 'Smith Darren'),
 ('Coplin', 'Joan', 'Baker Timothy'),
 ('Crumpet', 'Erica', 'Smith Tracy'),
 ('Dare', 'Nancy', 'Joplette Janice'),
 ('Farrell', 'David', None),
 ('Farrell', 'Jemima', None),
 ('Genting', 'Matthew', 'Butters Gerald'),
 ('Hunt', 'John', 'Purview Millicent'),
 ('Jones', 'David', 'Joplette Janice'),
 ('Jones', 'Douglas', 'Jones David'),
 ('Joplette', 'Janice', 'Smith Darren'),
 ('Mackenzie', 'Anna', 'Smith Darren'),
 ('Owen', 'Charles', 'Smith Darren'),
 ('Pinker', 'David', 'Farrell Jemima'),
 ('Purview', 'Millicent', 'Smith Tracy'),
 ('Rownam', 'Tim', None),
 ('Rumney', 'Henrietta', 'Genting Matthew'),
 ('Sarwin', 'Ramnaresh', 'Bader Florence'),
 ('Smith', 'Darren', None),
 ('Smith', 'Darren', None),
 ('Smith', 'Jack', 'Smith Darren'),
 ('Smith', 'Tracy', None),
 ('Stibbons', 'Ponder', 'Tracy Bu

Q12: Find the facilities with their usage by member, but not guests

In [78]:
query="""SELECT name,count(memid) as usage_by_member
FROM Bookings
INNER JOIN Facilities
USING (facid)
WHERE memid <> 0
GROUP BY name
ORDER BY usage_by_member DESC
"""
rs = con.execute(query)
print(rs.keys())
rs.fetchall()

['name', 'usage_by_member']


[('Pool Table', 783),
 ('Snooker Table', 421),
 ('Massage Room 1', 421),
 ('Table Tennis', 385),
 ('Badminton Court', 344),
 ('Tennis Court 1', 308),
 ('Tennis Court 2', 276),
 ('Squash Court', 195),
 ('Massage Room 2', 27)]

All 9 facilities were used by members, the Pool Table was the most popular and used 783 times. The Massage Room 2 was the least popular.

Q13: Find the facilities usage by month, but not guests

In [85]:
query="""SELECT name, strftime('%m', starttime) as month, count(memid) as usage_by_member
FROM Bookings
INNER JOIN Facilities
USING (facid)
WHERE memid <> 0
GROUP BY name,month
ORDER BY usage_by_member DESC
"""
rs = con.execute(query)
print(rs.keys())
rs.fetchall()

['name', 'month', 'usage_by_member']


[('Pool Table', '09', 408),
 ('Pool Table', '08', 272),
 ('Snooker Table', '09', 199),
 ('Table Tennis', '09', 194),
 ('Massage Room 1', '09', 191),
 ('Badminton Court', '09', 161),
 ('Snooker Table', '08', 154),
 ('Massage Room 1', '08', 153),
 ('Table Tennis', '08', 143),
 ('Badminton Court', '08', 132),
 ('Tennis Court 1', '09', 132),
 ('Tennis Court 2', '09', 126),
 ('Tennis Court 1', '08', 111),
 ('Tennis Court 2', '08', 109),
 ('Pool Table', '07', 103),
 ('Squash Court', '09', 87),
 ('Squash Court', '08', 85),
 ('Massage Room 1', '07', 77),
 ('Snooker Table', '07', 68),
 ('Tennis Court 1', '07', 65),
 ('Badminton Court', '07', 51),
 ('Table Tennis', '07', 48),
 ('Tennis Court 2', '07', 41),
 ('Squash Court', '07', 23),
 ('Massage Room 2', '09', 14),
 ('Massage Room 2', '08', 9),
 ('Massage Room 2', '07', 4)]

The pool table had the highest monthly rate by member, it was booked 408 times in September 2012 (the data cover 3 months in 2012 - Jul, Aug, and Sep).